In [ ]:
import pandas as pd

# The Cover Type Data Set

You can download it here:

https://archive.ics.uci.edu/ml/datasets/Covertype

In [ ]:
# Column names for the data, c.f. the description for details.
# One important thing to note is that some of the variables
# (e.g. the soil type) are indicator variables. One could have
# a discussion about how sensible it is to use LDA with these,
# but we'll just proceed for educational value.
column_names = (
["Elevation",
"Aspect",
"Slope",
"Horizontal_Distance_To_Hydrology",
"Vertical_Distance_To_Hydrology",
"Horizontal_Distance_To_Roadways",
"Hillshade_9am",
"Hillshade_Noon",
"Hillshade_3pm",
"Horizontal_Distance_To_Fire_Points"]
    + ['WE{}'.format(i) for i in range(4)]
    + ['ST{}'.format(i) for i in range(40)]
    + ['Cover'])

In [ ]:
cover_data = pd.read_csv('data/covtype.data.gz', names=column_names)

In [ ]:
cover_data.head()

In [ ]:
%matplotlib inline

In [ ]:
# Some variables look somewhat Gaussian, or
# at least like on could transform them to
# look somwhat Gaussian...
cover_data.Elevation.plot.hist(bins=30)

In [ ]:
# ... while some clearly don't
cover_data.WE2.hist()

In [ ]:
# Let's look at the priors to make sure that we
# don't have a singular, very small class.
cover_data.groupby('Cover').Cover.count() / cover_data.shape[0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sample = cover_data.sample(10000)

In [ ]:
# plotting two examplary variables against each other
# shows that the classification task is a hard one
for k, group in sample.groupby('Cover'):
    plt.scatter(group.Elevation, group.Slope, alpha=0.5)

# Fitting Linear Discriminant Analysis

We now proceed to fit a LDA to our data.

$$\begin{align}
\hat \pi_l &= \frac{N_l}{N}\\
\hat \mu_l &= \frac 1 {N_l} \sum_{i, g_i = l} x_i\\
\hat \Sigma &= \frac 1 {N - K} \sum_l \sum_{i, g_i = l} (x_i - \hat \mu_l)(x_i -
  \hat \mu_l)^T\\
\hat \Sigma_l &= \frac 1 {N_l - 1} \sum_{i,g_i = l}(x_i - \hat \mu_l)(x_i -
  \hat \mu_l)^T
\end{align}$$

In [ ]:
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(cover_data, test_size=0.3)

In [ ]:
# Sanity check: Make sure we have all the classes
# in the training data set ...
train.groupby('Cover').Cover.count()

In [ ]:
# ... and in the test data set as well...
test.groupby('Cover').Cover.count()

In [ ]:
means, pis, labels, Sigmas = [], [], [], []
N, p = train.shape
p -= 1
Sigma = np.zeros([p, p])
for label, data in train.groupby('Cover'):
    labels.append(label)
    Nk, _ = data.shape
    # prior
    pi = Nk / float(N)
    pis.append(pi)
    # mean
    mu = data.mean()
    # below, XXX[:-1]: Drop the 'Cover' column
    means.append(mu[:-1])
    xn = (data - mu).values[:,:-1]
    # Sigma_k
    S = np.zeros([p, p])
    for i in range(Nk):
        S += np.dot(xn[i:i+1,:].T, xn[i:i+1,:])
    Sigmas.append(S / (Nk - 1))
    # total Sigma
    Sigma += S
# normalize Sigma
Sigma /= float(N - len(labels))

In [ ]:
# Don't try this at home.
# It's usually a bad idea to invert a matrix!
# Recommended reading: https://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix/
Sigmainv = np.linalg.inv(Sigma)

In [ ]:
pis

In [ ]:
# Discriminant functions
def delta(x, mu, pi):
    return (np.dot(np.dot(x, Sigmainv), mu)
            - 0.5 * np.dot(np.dot(mu.T, Sigmainv), mu)
            + np.log(pi))

In [ ]:
# calssifer
def lda(x):
    return np.argmax(np.array([delta(x, mu, pi) for mu, pi in zip(means, pis)]).T, axis=1)
# Note that lda classifies 0, ..., K-1 ...

In [ ]:
# ... but our labels are 1, ..., K
labels

In [ ]:
# So we need to add 1 to make a prediciton
# Let's calcuate the hit rate:
# HR = (true positives + true negatives) / N
# Note we need column_names[:-1] to drop the 'Cover' column
((lda(test[column_names[:-1]]) + 1) == test.Cover).mean()

In [ ]:
# per-class hit rate:
for k, data in test.groupby('Cover'):
    print k, (lda(data[column_names[:-1]]) + 1 == data.Cover).mean()